## 找出 cost 最低的一天

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
from dotenv import load_dotenv
import seaborn as sns
from datetime import time
import pandas as pd
from collections import defaultdict
import json
warnings.filterwarnings('ignore')
load_dotenv()

True

## Real Time Price

In [3]:
# set start and end datetime
start_datetime_str = '2018-09-01'
end_datetime_str = '2018-09-30'

# convert string to datetime
start_datetime = datetime.strptime(start_datetime_str, '%Y-%m-%d')
end_datetime = datetime.strptime(end_datetime_str, '%Y-%m-%d')


def set_real_time_price_range(real_time_price, start_time: datetime, end_time: datetime):
    real_time_price = real_time_price[(real_time_price['datetime'] >= start_time) & (real_time_price['datetime'] <= end_time)].copy()
    real_time_price.sort_values(by='datetime', inplace=True)
    return real_time_price

# get real time price data
real_time_price = pd.read_csv('./Dataset/RTP/electricity_prices_from_201807010000_to_201812312359.csv')
real_time_price['datetime'] = pd.to_datetime(real_time_price['datetime'])
real_time_price = set_real_time_price_range(real_time_price, start_datetime, end_datetime)
real_time_price

,datetime,average_price
1487,2018-09-01 00:00:00,0.021917
1488,2018-09-01 01:00:00,0.020750
1489,2018-09-01 02:00:00,0.020083
1490,2018-09-01 03:00:00,0.020000
1491,2018-09-01 04:00:00,0.019917
...,...,...
2179,2018-09-29 20:00:00,0.014333
2180,2018-09-29 21:00:00,0.016333
2181,2018-09-29 22:00:00,0.018583
2182,2018-09-29 23:00:00,0.012750


In [17]:
# Load your data from CSV files
data = pd.read_csv('./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/test_building_loading_history.csv')
prices = pd.read_csv('./Dataset/RTP/electricity_prices_from_201807010000_to_201812312359.csv')

# Convert the time columns to datetime format
data['current_time'] = pd.to_datetime(data['current_time'])
prices['time'] = pd.to_datetime(prices['datetime'])

# # Merge the data and price files based on time
merged_data = pd.merge_asof(data.sort_values('current_time'), prices.sort_values('time'), 
                            left_on='current_time', right_on='time', direction='backward')

# # Calculate the cost for each time period
merged_data['cost'] = merged_data['total_load'] * merged_data['average_price']

# # Group by day and calculate total daily cost
daily_cost = merged_data.groupby(merged_data['current_time'].dt.date)['cost'].sum()

# # Find the day with the lowest total cost
min_cost_day = daily_cost.idxmin()
min_cost_value = daily_cost.min()

# # Print the daily total costs
print("Daily Total Costs:")
print(daily_cost)

# # Print the day with the lowest total cost
print(f"\nDay with the lowest total cost: {min_cost_day} with a total cost of {min_cost_value}")

Daily Total Costs:
current_time
2018-09-01     92.226388
2018-09-02     76.135549
2018-09-03    219.026167
2018-09-04    236.593428
2018-09-05    211.340363
2018-09-06    189.061890
2018-09-07    194.063132
2018-09-08     78.500589
2018-09-09     52.943404
2018-09-10    162.144867
2018-09-11    211.629844
2018-09-12    305.061272
2018-09-13    295.989103
2018-09-14    330.223870
2018-09-15    180.016851
2018-09-16    112.130866
2018-09-17    406.911663
2018-09-18    242.438920
2018-09-19    369.605702
2018-09-20    352.727439
2018-09-21    173.457115
2018-09-22     53.994374
2018-09-23     77.912515
2018-09-24    215.045088
2018-09-25    235.265541
2018-09-26    186.767146
2018-09-27    174.543880
2018-09-28    189.022710
2018-09-29     56.155220
2018-09-30     48.907654
Name: cost, dtype: float64

Day with the lowest total cost: 2018-09-30 with a total cost of 48.90765352767561


In [42]:

# Load the data
data = pd.read_csv('./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/test_building_loading_history.csv')
soc_data = pd.read_csv('./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/test_soc_history.csv')

# Convert the time columns to datetime format
data['current_time'] = pd.to_datetime(data['current_time'])
soc_data['timestamp'] = pd.to_datetime(soc_data['current_time'])

# Filter the data for 9/30
date_str = '2018-09-30'
data_0930 = data[data['current_time'].dt.date == pd.to_datetime(date_str).date()]
soc_data_0930 = soc_data[soc_data['timestamp'].dt.date == pd.to_datetime(date_str).date()]

# Initialize the dictionary to store the output
output = defaultdict(dict)

# Create a mapping of requestID to EV names
request_to_ev = {f"2018093000{i}": f"EV{i}" for i in range(1, 11)}

# Loop through each hour and fill in the values
for index, row in data_0930.iterrows():
    hour = row['current_time'].strftime('%H:%M')
    output[hour]["price"] = prices[prices['time'] == row['current_time']]['average_price'].values[0]
    output[hour]["total_EV_consumption"] = row['total_action_impact']
    output[hour]["building_consumption"] = row['original_load']
    
    ev_soc = {}
    for _, soc_row in soc_data_0930[soc_data_0930['timestamp'] == row['current_time']].iterrows():
        ev_name = request_to_ev.get(str(soc_row['requestID']), None)
        if ev_name:
            ev_soc[ev_name] = soc_row['soc']
    
    output[hour]["EV_detail_SoC"] = ev_soc

# Convert the dictionary to a JSON-like format and print the result
formatted_output = json.dumps(output, indent=4)

# Save the formatted output to a JSON file
with open('./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/output_0930.json', 'w') as json_file:
    json_file.write(formatted_output)

print("Output saved to ./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/output_0930.json")

Output saved to ./Result/GB-MARL-v2.1_alpha0.5_beta0.1_num10_s1/23/output_0930.json
